In [ ]:
%load_ext autoreload
%autoreload 2


import numpy as np
import time, os, sys

import skimage.io
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from scipy.ndimage import binary_closing,grey_closing
from skimage import measure
import napari 
import omnipose
import ncolor 
from omnipose.utils import sinebow

from scipy.ndimage import gaussian_filter
from omnipose.utils import normalize99, rescale
from scipy.ndimage import gaussian_filter

def localnormalize(im,sigma1=2,sigma2=3):
    im = normalize99(im)
    blur1 = gaussian_filter(im,sigma=sigma1)
    num = im - blur1
    blur2 = gaussian_filter(num*num, sigma=sigma2)
    den = np.sqrt(blur2)
    
    return normalize99(num/den+1e-8)

def cyclic_perm(a):
    n = len(a)
    b = [[a[i - j] for i in range(n)] for j in range(n)]
    return b

# Make a function that extracts the file name 
from pathlib import Path
def getname(path,suffix=''):
    return os.path.splitext(Path(path).name)[0].replace(suffix,'')

import tifffile
from omnipose import core
import fastremap
import scipy
from cellpose_omni import io

In [ ]:
from aicsimageio import AICSImage

# Get an AICSImage object
number = 29
img = AICSImage('/home/kcutler/DataDrive/3D_BBBC/BBBC024_v1_c75_highSNR_images/image-final_'+"%04d" % (number,)+'.ics').data.squeeze()
mask =  AICSImage('/home/kcutler/DataDrive/3D_BBBC/BBBC024_v1_c75_highSNR_foreground/image-labels_'+"%04d" % (number,)+'.ics').data.squeeze()

In [ ]:
img.shape

In [ ]:
from cellpose_omni import io
save0 = '/home/kcutler/DataDrive/3D_BBBC/BBBC027'
savedir = []
cpdir = []
ext = '.tif'
for t in ['test','train']:
    sd = os.path.join(save0,t)
    io.check_dir(sd)
    savedir.append(sd)
    sd = os.path.join(save0,'cellpose',t)
    io.check_dir(sd)
    cpdir.append(sd)
    
# from cellpose_omni import io
# save0 = '/home/kcutler/DataDrive/3D_BBBC/BBBC027'
# export = '/home/kcutler/DataDrive/3D_BBBC/BBBC027'

# io.check_dir(export)
# savedir = []
# expdir = []
# cpdir = []
# ext = '.tif'
# for t in ['test','train']:
#     sd = os.path.join(save0,t)
#     savedir.append(sd)
#     sd = os.path.join(export,t)
#     io.check_dir(sd)
#     expdir.append(sd)
#     sd = os.path.join(export,'cellpose',t)
#     io.check_dir(sd)
#     cpdir.append(sd)
# savedir,expdir,cpdir


In [ ]:
import fastremap
for number in range(30):
    name = "%04d" % (number,)
    img = AICSImage(os.path.join(save0,'image-final_'+name+'.ics')).data.squeeze()
    mask =  AICSImage(os.path.join(save0,'image-labels_'+name+'.ics')).data.squeeze()
    labels, _ = scipy.ndimage.label(mask)
    labels = fastremap.refit(labels)
    if number%3:
        sd = savedir[0]
    else:
        sd = savedir[1]
        
    tifffile.imwrite(os.path.join(sd,name+ext),img)
    tifffile.imwrite(os.path.join(sd,name+'_masks'+ext),labels)

In [ ]:

d = 3
idx = np.arange(d)
c = np.array([0]*(d-2)+[1]*2)
cyclic_perm(c),idx,c
ext = '.tif'
a = 'zyx'
# for basedir,save0 in zip(expdir,cpdir):
for basedir,save0 in zip([savedir[1]],[cpdir[1]]): #only do train 

    mask_filter = '_masks'
    img_names = io.get_image_files(basedir,mask_filter)
    mask_names,_ = io.get_label_files(img_names, mask_filter)
    for p1,p2 in zip(img_names,mask_names):
        img = tifffile.imread(p1)
        mask = tifffile.imread(p2)
        s = img.shape
        name = getname(p1)
        for inds,i in zip(cyclic_perm(c),idx):
            for k in range(s[i]):
                slc = tuple([slice(-1) if i else k for i in inds])
                suffix = '_'+a[i]+'{:03d}'.format(k)
                l = omnipose.utils.format_labels(mask[slc],clean=True)
                if np.any(l): # only keep slices that have masks
                    tifffile.imwrite(os.path.join(save0,name+suffix+ext),np.uint16(omnipose.utils.rescale(img[slc])*(2**16-1)))
                    tifffile.imwrite(os.path.join(save0,name+suffix+'_masks'+ext),l)

In [ ]:
img[slc].shape

In [ ]:
import fastremap
labels = fastremap.refit(labels)
labels.dtype

In [ ]:
import ncolor
from skimage import measure
# labels = ncolor.label(measure.label(mask))
import scipy
labels, _ = scipy.ndimage.label(mask)
# labels = ncolor.label(labels)
viewer = napari.view_image(img, name='cells')
viewer.add_labels(labels, name='labels')

In [ ]:
labels.max()

In [ ]:
labels[0].max()

In [ ]:
!pip install b3get

In [ ]:
!b3get list

In [ ]:
import b3get

images, labels = b3get.to_numpy(27)

In [ ]:
k = 0
viewer = napari.view_image(images[k], name='cells')
viewer.add_labels(labels[k], name='labels')

In [ ]:
images[1].shape

In [ ]:
masks32 = tifffile.imread('/home/kcutler/DataDrive/3D_BBBC/BBBC032_v1_DatasetGroundTruth.tif')

In [ ]:
c0 = tifffile.imread('/home/kcutler/DataDrive/3D_BBBC/BBBC032_v1_dataset/BMP4blastocystC0.tif')
c1 = tifffile.imread('/home/kcutler/DataDrive/3D_BBBC/BBBC032_v1_dataset/BMP4blastocystC1.tif')
c2 = tifffile.imread('/home/kcutler/DataDrive/3D_BBBC/BBBC032_v1_dataset/BMP4blastocystC2.tif')
c3 = tifffile.imread('/home/kcutler/DataDrive/3D_BBBC/BBBC032_v1_dataset/BMP4blastocystC3.tif')

In [ ]:
viewer = napari.view_image(c0, name='c0',colormap='green',opacity=.5,blending='additive')
viewer.add_image(c1, name='c1',colormap='gray',opacity=.5,blending='additive')
viewer.add_image(c2, name='c2',colormap='blue',opacity=.5,blending='additive')
viewer.add_image(c3, name='c3',colormap='red',opacity=.5,blending='additive')

viewer.add_labels(masks32, name='labels')